# PyTorch своими руками

In [2]:
# только numpy, только хардкор
import numpy as np

Слои — это такие элементарные блоки, из которых состоят нейросети. Конечным пользователям вашего фреймворка не хочется думать, как они работают. Им просто хочется объевить последовательность операций над входными данными, а о градиентах и прочем матане пусть фреймворк позаботится.

**Module** — это абстрактный класс, от которого будут наследоваться слои нашей нейронной сети. Абстрактные классы нужны, чтобы можно было реализовывать не все методы, а только переопределить некоторые. Все в лучших традициях ООП.

Модуль — это такая чёрная коробка, которая
1. Умеет принимать какие-то входные данные $X$ и возращать какие-то выходные данные $Y$ (`forward`)
2. Возможно, имеет какие-то параметры, которые можно изменять, (`parameters`, `grad_parameters`)
3. Будучи встроенной в вычислительный граф, умеет по градиенту относительно своих выходных значений вычислять градиент относительно входных данных, а также собственных параметров (`backward`)
4. Умеет переключаться в режимы обучения и инференса, если они отличаются (`train`, `eval`)

Теперь поподробнее.

## Входные данные

Слой можно рассматривать как функцию, которая принимает и возвращает вектора из действительных чисел.

Современный нейросети оптимизируют различными вариантами *стохастического градиентного спуска*, и мы тоже будем его использовать. Его отличие от обычного в том, что для оценки градиента на каждом шаге рассматривается не весь датасет, а лишь его маленькая часть, которую называют батчем (`batch`). Если батч формировать случайно, и если его размер достаточно большой, то мы можем быстро получить немного шумную, но примлимую оценку градиента, а не прогонять через сеть все миллионы примеров ради одного шажка. Эта интуиция ограничивает размер батча сверху.

Математик бы принял время прогона одного примера по всей сети за константу и пришел бы к выводу, что нужно считать по одному примеру и делать каждый раз один шаг, но маленький. Это верное заключение, но в реальности, если увеличить размер батча в $k$ раз, то он будет работать не в $k$ раз дольше, а намного меньше.

Самая долгая операция в большинстве нейросетей — это перемножение матриц. Начиная с каких-то размеров матриц для их перемножения имеет ммысл использовать алгоритм Штрассена, который работает уже быстрее, чем линейно. Проведем небольшой вычислительный эксперимент.

In [47]:
A = np.random.randn(256, 2000)
B = np.random.randn(2000, 800)

%time C = np.stack(np.dot(A[i].T, B) for i in range(256))

%time C = np.dot(A, B)

CPU times: user 964 ms, sys: 57.1 ms, total: 1.02 s
Wall time: 263 ms
CPU times: user 164 ms, sys: 13 ms, total: 177 ms
Wall time: 45.5 ms


Такая чисто вычислительная причина ограничивает размер батча снизу. На практике, в большинстве случаев оптимальный размер батча — несколько сотен. В случае с CPU это несколько десятков, потому что выгода от распараллеливания вычислений не такая сильная.

Данные мы тоже будем подавать как матрицы, а не вектора. Вообще, более сложные нейросети по-другому кодируют свои данные (например, картинки — это четырехмерный тензор: `[batch, channel, x, y]`).

«тензор» это, вообще говоря, сложный математический объект, но в DL этот термин используется просто в занчении «многомерный массив».

В конце вычислительного графа всегда функция потерь, которую в итоге нужно минимизировать.

Параметр — это что-то, что можно обучать. Он должен быть доступен оптимизатору, а оптимизатор не должен знать, как всё у нас внутри работает. Поэтому градиенты считать тоже нужно нам.

### Forward

**Важно**: нам для `backward` будет намного проще сохранять `output`, посчитанный во время `forward`.

### Backward

Обратный прогон долен делать две вещи:

1. Посчитать и сложить

### train / eval

Некоторые слои ведут себя по-разному при обучении и реальных прогонах.

По сути, нужно просто написать два разных forward-а для обучения и инференса.

Например, `BatchNorm` и `DropOut`.

In [2]:
class Module():
    def __init__(self):
        self._train = True
    
    def forward(self, input):
        raise NotImplementedError

    def backward(self,input, grad_output):
        raise NotImplementedError
    
    def parameters(self):
        'Возвращает список собственных параметров.'
        return []
    
    def grad_parameters(self):
        'Возвращает список тензоров-градиентов для своих параметров.'
        return []
    
    def train(self):
        self._train = True
    
    def eval(self):
        self._train = False

Это **абстрактный класс** — от него наследуются другие, в которых есть настоящая реализация.

# Sequential

**Sequential** будет оборачивать список модулей и выполнять их последовательно.

Это своего рода контейнер, внутри которого есть какой-то пайплайн.

Можно даже засовывать один Sequential внутри другого.

Многие не знают, но в питоне почти всегда для итерирования используется не **deep copy**, а **shallow copy**. Это делается для экономии памяти.

In [3]:
class Sequential(Module):
    def __init__ (self, *layers):
        super().__init__()
        self.layers = layers

    def forward(self, input):
        """
        Прогоните данные последовательно по всем слоям:
        
            y[0] = layers[0].forward(input)
            y[1] = layers[1].forward(y_0)
            ...
            output = module[n-1].forward(y[n-2])   
            
        Это должен быть просто небольшой цикл: for layer in layers...
        
        Хранить выводы ещё раз не надо: вспомните -- они сохраняются внутри слоев после forward.
        """

        # ...
        
        return self.output

    def backward(self, input, grad_output):
        """
        Backward -- это как forward, только наоборот. (с)
        
        Предназначение backward — посчитать посчитать градиенты для собственных параметров и передать градиент относительно своего входа.
        
        Внутри параметров модули сами позаботятся о своих параметрах. Нам же нужно позаботиться о передачи градиента.
         
            g[n-1] = layers[n-1].backward(y[n-2], grad_output)
            g[n-2] = layers[n-2].backward(y[n-3], g[n-1])
            ...
            g[1] = layers[1].backward(y[0], g[2])   
            grad_input = layers[0].backward(input, g[1])
        
        Тут цикл будет уже посложнее.
        """
        # ...
        
        return grad_input
      
    def parameters(self):
        'Можно просто сконкатенировать все параметры в один список.'
        return [l.parameters() for l in self.layers]
    
    def grad_parameters(self):
        'Можно просто сконкатенировать все градиенты в один список.'
        return [l.grad_parameters() for l in self.layers]

# Слои

Приступим к реализации содержательной части — самих слоев.

Работать мы будем с векторами.

Все операции обучно формулируются так, что они не зависят друг от друга (кроме `BatchNorm`, но об этом позже).

Во всех фреймворках . Мы тоже будем привыкать к этому.

Начнем с основного: линейный слой. Он же афинный, он же fully-conected.

На вход всех слоев будет подаваться матрица размера `batch_size` $\times$ `n_features`.

In [4]:
class Linear(Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
       
        # на самом деле, очень важно, как оно инициализируется
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size=(dim_out, dim_in))
        self.b = np.random.uniform(-stdv, stdv, size=dim_out)
        
    def forward(self, input):
        # ...      
        return self.output
    
    def backward(self, input, grad_output):
        # ...
        return grad_input
    
    def parameters(self):
        return [self.W, self.b]
    
    def grad_parameters(self):
        return [self.grad_W, self.grad_b]

## Функции активации

**ReLU** — одна из самых простых функций активации:

$$
ReLU(x) = \begin{cases}

\end{cases}
$$

`ReLU` очень простой, поэтому автору не жалко его реализовать за вас:

In [8]:
class ReLU(Module):
    def __init__(self):
         super().__init__()
    
    def forward(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def backward(self, input, grad_output):
        grad_input = np.multiply(grad_output, input > 0)
        return grad_input

У ReLU есть проблема — у него бесполезная нулевая производная при $x < 0$.

Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope. 

In [9]:
class LeakyReLU(Module):
    def __init__(self, slope=0.03):
        super().__init__()
            
        self.slope = slope
        
    def forward(self, input):
        # ...
        return self.output
    
    def backward(self, input, grad_output):
        # ...
        return grad_input

Софтмакс — самый сложный с точки зрения написания `backward`. Хотя, как и все, занимает 5 строчек.

$$ SoftMax(x_k) = \frac{e^{x_k}}{\sum_{i=1}^n e^{x_i} }$$

In [ ]:
class SoftMax(Module):
    def __init__(self):
         super().__init__()
    
    def forward(self, input):
        # важная деталь: если входы большие,
        # то экспоненты будут ещё больше
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # ...
        return self.output
    
    def backward(self, input, grad_output):
        # ...
        return grad_input

## Регуляризация

Реализуйте [**дропаут**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). Идея простая: просто помножьте входные данные на случайную маску того же размера. Сгенерировать маску можно через `np.random.binomial`.

Дропаут клёвый, и его обычно хватает как единственного регуляризатора. Если вы заметите, что сеть оверфитится — просто добавьте его побольше.

Заметитьте, что у него разное поведение в `train` и `eval` режимах. При `eval` он не должен делать ничего, а в `train` помимо маски нужно ещё домножить вход на $\frac{1}{1-p}$, чтобы ожидание не изменилось.

In [ ]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super().__init__()
        
        self.p = p
        self.mask = None
        
    def forward(self, input):
        self.output = 
        mask = # ...
        self.output = # ...
        return self.output
    
    def backward(self, input, grad_output):
        # ...
        return grad_input

## Критерии

Критерии — это специальные функции, которые меряют качество, имея реальные данные и предсказанные.

По сути это тоже модули. Их следует отделять от модулей, котому что у них нет `train` / `eval`, а `backward` не требует `grad_output` — эта вершина и так конечная в вычислительном графе.

Также нам не понадобится сохранять output.

In [8]:
class Criterion():        
    def forward(self, input, target):
        raise NotImplementedError

    def backward(self, input, target):
        raise NotImplementedError

Реализуем $L_2$-норму: просто средняя квадратичная ошибка.

Обратите внимание, что в критериях мы делим итоговую цифру на размер батча — мы не хотим, чтобы функция потерь зависела от количества примеров.

In [5]:
import numpy as np

In [9]:
class MSE(Criterion):
    def forward(self, input, target):
        batch_size = input.shape[0]
        self.output = np.sum(np.power(input - target, 2)) / batch_size
        return self.output
 
    def backward(self, input, target):
        self.grad_output  = (input - target) * 2 / input.shape[0]
        return self.grad_output

Ваша задача посложнее: вам нужно реализовать кроссэнтропию — стандартная функция потерь для классификации.

Помните, что лэйблы в one-hot.

Напоминаем интуицию за принципом максимального правдоподобия: максимизируем произведение прсказанных вероятностей реальных событий $ L = \prod p_i $.

Произведение оптимизировать очень не удобно, и поэтому воспользуемся следующим трюком: возьмем логарифм (любой, ведь все логарифмы отличаются в константу раз) и будем максимизировать сумму:

$$ \log L = \log \prod p_i = \sum \log p_i $$

Эту штуку называют кроссэнтропией. Такое название пошло из теории информации, но нам пока знать это не надо.

Для удобноства вместо чисел — от 0 до 9 — сконвертируем их в вектора размера 10, где будет стоять единица в нужном месте (такая кодировка называется one-hot).

In [14]:
class CrossEntropy(Criterion):
    def __init__(self):
        super().__init__()
        
    def forward(self, input, target): 
        # можно сделать такой трюк, чтобы нигде не было взятий логарифма от нуля:
        eps = 1e-9
        input_clamp = np.clip(input, eps, 1 - eps)
        
        # ...
        return self.output

    def backward(self, input, target):
        # опять же, больба с численными ошибками
        input_clamp = np.maximum(1e-15, np.minimum(input, 1 - 1e-15) )
                
        # ....
        return grad_input